In [6]:
# Restart Kernel after install so modules can be imported
%cd /project_ghent/GANs_Pose_Estimation_on_Art_Collection
%sudo bash "commands/Style Transfer/install_CycleGAN.sh"
%pip install -r "requirements.txt"
%cd src

/project_ghent/GANs_Pose_Estimation_on_Art_Collection
Update pip
Looking in indexes: https://download.pytorch.org/whl/cu118
Already up to date.
running develop
/opt/conda/lib/python3.11/site-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/opt/conda/lib/python3.11/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid 

In [1]:
import sys
import os
sys.path.append(f"{os.getcwd()}")
import torch
import cv2
from PIL import Image
from torchvision.transforms import transforms
from style_transfer.networks import CycleGAN, CycleGANConfig
from CycleGAN.data.unaligned_dataset import UnalignedDataset

In [2]:
def train():
    config = CycleGANConfig.create("style_transfer/config/cyclegan_train.yaml", phase="train")
    
    config.defrost()
    config.dataroot = "../../../Datasets/custom/cezanne2photo"
    config.name = "cezanne"
    config.display_freq = 2
    config.print_freq = 2
    config.save_no = 1
    config.save_epoch_freq = 1
    config.continue_train = True
    config.display_server = "http://116.203.134.130"
    config.display_env = "test_mount"
    config.freeze()
    print(config)
    
    dataset = UnalignedDataset(config)
    
    network = CycleGAN(config)
    # network.loadModel("../../../Models/CycleGAN/checkpoints/")
    network.train(dataset)


In [2]:
def transform(path, direction="AtoB"):
    config = CycleGANConfig.create("style_transfer/config/cyclegan_test.yaml", phase="test")
    
    config.defrost()
    config.results_dir = "../.output/results"   # saves results here.
    config.num_threads = 0   # test code only supports num_threads = 0
    config.batch_size = 1    # test code only supports batch_size = 1
    config.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
    config.no_flip = True    # no flip; comment this line if results on flipped images are needed.
    config.display_id = -1   # no visdom display
    config.freeze()
    
    network = CycleGAN(config)
    network.loadModel({
        "G_A": "../../../Models/CycleGAN/monet_A_B.pth",
        "G_B": "../../../Models/CycleGAN/monet_B_A.pth"
    })
    with open(path, 'rb') as file:
        image = Image.open(file)
        image.convert("RGB")
    
    image = transforms.ToTensor()(image)
    image = transforms.Resize(256)(image)
    image = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(image)
    image = torch.stack((image,))
    image = image.to("cuda")

    if direction == "AtoB":        
        image = network.transformFromPhotographicToArtistic(image)
    elif direction == "BtoA":
        image = network.transformFromArtisticToPhotographic(image)
    
    image = image[0] * 0.5 + 0.5
    image = image.detach().cpu().numpy()
    image = image.transpose(1,2,0)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(config.results_dir, "test_cyclegan.png"), image * 255.0)

In [3]:
# transform("../../../Datasets/custom/Drama/8_1/000000001240.jpg", "AtoB")
train()

batch_size: 1
beta1: 0.5
checkpoints_dir: ../../../Models/CycleGAN/checkpoints
continue_train: True
crop_size: 256
dataroot: ../../../Datasets/custom/cezanne2photo
dataset_mode: unaligned
direction: AtoB
display_env: test_mount
display_freq: 2
display_id: 1
display_ncols: 4
display_port: 8097
display_server: http://116.203.134.130
display_winsize: 256
epoch: latest
epoch_count: 1
gan_mode: lsgan
gpu_ids: [0]
init_gain: 0.02
init_type: normal
input_nc: 3
isTrain: True
lambda_A: 10.0
lambda_B: 10.0
lambda_identity: 0.5
load_size: 286
lr: 0.0002
lr_decay_iters: 50
lr_policy: linear
max_dataset_size: inf
model: train
n_epochs: 100
n_epochs_decay: 100
n_layers_D: 3
name: cezanne
ndf: 64
netD: basic
netG: resnet_9blocks
ngf: 64
no_dropout: False
no_flip: True
norm: instance
num_threads: 4
output_nc: 3
phase: train
pool_size: 50
preprocess: resize_and_crop
print_freq: 2
save_epoch_freq: 1
save_no: -1
serial_batches: True
suffix: 
verbose: True
initialize network with normal
initialize network

Setting up a new session...
c:\Users\tverh\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


learning rate 0.0002000 -> 0.0002000
(epoch: 21, iters: 2, time: 1.151, data: 2.858) D_A: 1.355 G_A: 2.071 cycle_A: 1.876 idt_A: 1.671 D_B: 1.299 G_B: 1.673 cycle_B: 3.995 idt_B: 0.885 
(epoch: 21, iters: 4, time: 1.143, data: 0.071) D_A: 0.461 G_A: 0.652 cycle_A: 3.784 idt_A: 1.152 D_B: 0.758 G_B: 0.729 cycle_B: 2.747 idt_B: 1.900 
saving the model at the end of epoch 21, iters 5
End of epoch 21 / 200 	 Time Taken: 15 sec
learning rate 0.0002000 -> 0.0002000
(epoch: 22, iters: 1, time: 1.278, data: 0.056) D_A: 0.252 G_A: 0.638 cycle_A: 2.129 idt_A: 0.883 D_B: 0.383 G_B: 0.660 cycle_B: 2.559 idt_B: 1.053 
(epoch: 22, iters: 3, time: 1.106, data: 0.072) D_A: 0.500 G_A: 0.520 cycle_A: 2.822 idt_A: 1.096 D_B: 0.582 G_B: 0.740 cycle_B: 2.280 idt_B: 1.313 
(epoch: 22, iters: 5, time: 1.999, data: 0.054) D_A: 0.292 G_A: 0.537 cycle_A: 1.885 idt_A: 1.088 D_B: 0.360 G_B: 0.382 cycle_B: 2.524 idt_B: 0.850 
saving the model at the end of epoch 22, iters 10
End of epoch 22 / 200 	 Time Taken: 9 s

KeyboardInterrupt: 